In [1]:
## nli dataset load
## nli model load
## locate utils 불러와서, nli model을 backprop했을 때 locate되는 instance 분석 (random 30개)


from datasets import load_dataset


nli_dataset = load_dataset('stanfordnlp/snli')

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

nli_model = AutoModelForSequenceClassification.from_pretrained("ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli")
nli_tokenizer = AutoTokenizer.from_pretrained("ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli")


/data/hyeryung/.conda/envs/loc-edit/lib/python3.8/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/data/hyeryung/.conda/envs/loc-edit/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task

In [3]:
from new_module.locate.new_locate_utils import LocateMachine

class Args:
    task = 'nli'

locator = LocateMachine(nli_model, nli_tokenizer, Args())

In [4]:
## get test examples
import random

contradiction_indexes = [i for i, x in enumerate(nli_dataset['test']['label']) if x == 2]

random_indexes = random.sample(contradiction_indexes, 30)

test_examples = nli_dataset['test'][random_indexes]

test_examples_tuples = list(zip(*(test_examples['premise'], test_examples['hypothesis'])))

In [22]:
for example in test_examples_tuples:
    
    print("Premise:", example[0])
    print("Hypothesis:", example[1])
    print("\n") 

Premise: Man and woman rowing on surfboards third rower in distance.
Hypothesis: Couple going fishing.


Premise: The little boy is smiling as he crosses a rope on an assault course.
Hypothesis: The girl tries on the blue dress at the store.


Premise: Men sitting on the side of a brick road.
Hypothesis: the side of the road is empty.


Premise: Stick figures in the middle of the street.
Hypothesis: A picture of the ocean


Premise: A woman with a blond ponytail and a white hat is riding a white horse, inside a fence with a horned cow.
Hypothesis: The horse is being ridden by a man.


Premise: A man cooking with fire in like 5 pots at the same time!
Hypothesis: A man is cooking with a bunch of ovens.


Premise: People with bags walk along a dirt road in front of a green building.
Hypothesis: People are riding a bus.


Premise: While traveling, a woman in a black jacket is resting on the shoulder of a man in a black jacket.
Hypothesis: A man rests on a womans shoulder


Premise: Four me

In [5]:
## sample에 대한 output score 확인
tokens = nli_tokenizer(test_examples_tuples, return_tensors='pt', padding=True, truncation=True)

In [6]:
outputs = nli_model(**tokens)

In [7]:
import torch
probas = torch.softmax(outputs.logits, dim=1)[:,2]

In [10]:
probas

tensor([0.9769, 0.9991, 0.9970, 0.5598, 0.9971, 0.9994, 0.9960, 0.9993, 0.9718,
        0.9995, 0.9985, 0.8048, 0.9985, 0.9995, 0.9985, 0.9055, 0.9996, 0.9996,
        0.9838, 0.9984, 0.9654, 0.9993, 0.2090, 0.9963, 0.9970, 0.9879, 0.0330,
        0.9990, 0.9995, 0.9980], grad_fn=<SelectBackward0>)

In [6]:
nli_tokenizer.sep_token_id

2

In [8]:
result = locator.locate_main(test_examples_tuples, 'grad_norm', max_num_tokens = 1, unit="word", label_id=0)

prediction [('An entertainers reaches out to his audience during a performance.', 'A man juggling flaming torches.'), ('a red-haired woman playing the violin', 'A woman holds a broken violin.'), ('Two people standing behind a Sonoma display and a woman with a stroller on the other side.', 'Two people are race car driving.'), ('A man in a blue jacket leans on a post and talks on the phone while an old man carries yellow bags.', "Chew Wrigley's Spearmint beer."), ('Someone with a long pole that appears to have a spike on the end is pointing it at a red balloon on the other side of the wall from him.', 'A boy sleeping in a hot air balloon.'), ('Six older women are standing in a row at a table smiling and looking trough books and other material displayed on the table.', 'A man and a young boy are playing a game on a quilt on the floor.'), ('Two people are riding bicycles next to a building.', 'A woman is riding a unicycle past a building.'), ('Three men are playing soccer, two of them in b

IndexError: index 0 is out of bounds for dimension 0 with size 0

In [19]:
## observation 1: 어떤 sample들은 그닥 contradictive하지 않은 것 같다. --> 거를 방식이 있을까?
## observation 2: locate된 token들이 contradictive한 부분을 잘 반영하고 있는 것 같다. 

for i, ((p, h), y) in enumerate(zip(test_examples_tuples, result)):
    print("Premise: ", p)
    print("Hypothesis: ", h)
    print("Model proba: ", probas[i].item(), "->", probas_after_masking[i].item())
    print("Located p: ", y.split('</s></s>')[0])
    print("Located h: ", y.split('</s></s>')[1])
    print("\n")

Premise:  A woman is holding her child while a man is reading something from a piece of paper and taking something from a bowl that a younger girl is holding in a park with a small lake in the background.
Hypothesis:  The people are swimming in the lake.
Model proba:  0.9769063591957092 -> 0.764151930809021
Located p:  <s>A woman is holding her child while a man is reading something from a piece of paper and taking something from a bowl that a younger girl is holding in a park with a small lake in the background.
Located h:  The people are<mask> in the lake.</s>


Premise:  Two men in black jackets behind a white handrail.
Hypothesis:  Two women are wearing red
Model proba:  0.9990848302841187 -> 0.9990994930267334
Located p:  <s>Two men in black<mask> behind a white handrail.
Located h:  Two women are wearing red</s>


Premise:  People dresses in business clothing are walking by while one man with a tie on is standing still and talking on his cellular phone.
Hypothesis:  The people ar

In [13]:
## masking 된 샘플에 대한 energy score 확인
tokens_after_masking = nli_tokenizer(result, add_special_tokens=False, return_tensors='pt', padding=True, truncation=True)

In [14]:
probas_after_masking = nli_model(**tokens_after_masking).logits

In [16]:
probas_after_masking = torch.softmax(probas_after_masking, dim=-1)[:, 2]

In [17]:
probas_after_masking

tensor([0.7642, 0.9991, 0.0926, 0.2026, 0.9993, 0.9994, 0.8601, 0.8564, 0.7032,
        0.9994, 0.0068, 0.3111, 0.9986, 0.9995, 0.9035, 0.9318, 0.9996, 0.9984,
        0.1789, 0.9987, 0.1873, 0.1021, 0.0594, 0.1672, 0.0333, 0.0588, 0.0156,
        0.9992, 0.9995, 0.8343], grad_fn=<SelectBackward0>)

In [35]:
result2 = locator.locate_main(test_examples_tuples, 'grad_norm', max_num_tokens = 1, unit="word", label_id=2)

prediction [('Man and woman rowing on surfboards third rower in distance.', 'Couple going fishing.'), ('The little boy is smiling as he crosses a rope on an assault course.', 'The girl tries on the blue dress at the store.'), ('Men sitting on the side of a brick road.', 'the side of the road is empty.'), ('Stick figures in the middle of the street.', 'A picture of the ocean'), ('A woman with a blond ponytail and a white hat is riding a white horse, inside a fence with a horned cow.', 'The horse is being ridden by a man.'), ('A man cooking with fire in like 5 pots at the same time!', 'A man is cooking with a bunch of ovens.'), ('People with bags walk along a dirt road in front of a green building.', 'People are riding a bus.'), ('While traveling, a woman in a black jacket is resting on the shoulder of a man in a black jacket.', 'A man rests on a womans shoulder'), ('Four men wearing orange vests are putting in a steel beam on the ground.', 'Nobody is in orange'), ('A group of women obse

In [36]:
result2

['<s>Man and woman rowing on<mask>boards third rower in distance.</s></s>Couple going fishing.</s>',
 '<s>The little boy is smiling as he crosses a rope on an assault course.</s></s>The girl<mask> on the blue dress at the store.</s>',
 '<s>Men sitting on the side of a brick road.</s></s>the side of the road is<mask>.</s>',
 '<s>Stick figures in the middle of the street.</s></s>A picture of the<mask></s>',
 '<s>A woman with a blond ponytail and a white<mask> is riding a white horse, inside a fence with a horned cow.</s></s>The horse is being ridden by a man.</s>',
 '<s>A man cooking with fire in like 5 pots at the same time!</s></s>A man is cooking with a bunch of<mask>s.</s>',
 '<s>People with bags walk along a<mask> road in front of a green building.</s></s>People are riding a bus.</s>',
 '<s>While traveling, a woman in a black jacket is resting on the shoulder of a man in a black jacket.</s></s>A man rests on a<mask>ans shoulder</s>',
 '<s>Four men wearing orange vests are putting in

In [27]:

import numpy as np

[0.75156066 0.53843395 0.93593076 0.13970348 0.58124188 0.52094283
 0.66258606 0.46203155 0.45242672 0.2694994  0.26547587 0.08580285
 0.12868575 0.11124022 0.15886033 0.1278937  0.04750872 0.76374614
 0.62716798 0.47231046 0.63237988 0.37678597 0.38271456 0.26724505
 0.1582123  0.06426351 0.57355015 0.57597497 0.63484014 0.30908522]


In [32]:
for _ in range(30):
    a_list = np.random.rand(30)
    # print(a_list)

    # print("Average: ", np.mean(a_list))
    # print("Above average elements : ", a_list[a_list >= np.mean(a_list)])
    print("| Above average elements | (before exp): ", len(a_list[a_list >= np.mean(a_list)]))

    exp_a_list = np.exp(a_list)
    # print(exp_a_list)

    display_list = exp_a_list
    # print("Average: ", np.mean(display_list))
    # print("Above average elements : ", display_list[display_list >= np.mean(display_list)])
    print("| Above average elements | : (after exp)", len(display_list[display_list >= np.mean(display_list)]))
    
    if len(a_list[a_list >= np.mean(a_list)]) == len(display_list[display_list >= np.mean(display_list)]):
        print("same")
    else:
        print("different")
    print('-'*50)

| Above average elements | (before exp):  15
| Above average elements | : (after exp) 14
different
--------------------------------------------------
| Above average elements | (before exp):  15
| Above average elements | : (after exp) 15
same
--------------------------------------------------
| Above average elements | (before exp):  14
| Above average elements | : (after exp) 12
different
--------------------------------------------------
| Above average elements | (before exp):  15
| Above average elements | : (after exp) 15
same
--------------------------------------------------
| Above average elements | (before exp):  15
| Above average elements | : (after exp) 15
same
--------------------------------------------------
| Above average elements | (before exp):  16
| Above average elements | : (after exp) 14
different
--------------------------------------------------
| Above average elements | (before exp):  17
| Above average elements | : (after exp) 16
different
----------------

In [36]:
## torch where
import torch
input_tensor = torch.Tensor([[1,1,3,3,1],[1,1,1,3,1],[3,1,1,1,1]])



In [38]:
indices = (input_tensor == 3).nonzero(as_tuple=False)

In [39]:
first_occurence = torch.zeros(input_tensor.size(0), dtype=torch.long)
for i in range(input_tensor.size(0)):
    row_indices = indices[indices[:, 0]==i]
    if len(row_indices) > 0:
        first_occurence[i] = row_indices[0,1]


In [40]:
first_occurence

tensor([2, 3, 0])

In [42]:
mask = torch.zeros_like(input_tensor).long()
print(mask)

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])


In [45]:

for i in range(mask.size(0)):
    mask[i, :first_occurence[i]] = 1

In [46]:
mask

tensor([[1, 1, 0, 0, 0],
        [1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0]])

In [49]:
current_sent = '<s>Man and woman rowing on<mask>boards third rower in distance.</s></s>Couple going fishing.</s>'
words_ = current_sent.strip().split()
words = []
for w in words_:
    if ('<s>' in w):
        w = ['<s>', w.replace('<s>', '')]
    elif ('</s></s>' in w):
        w = w.split('</s></s>')
        w.insert(1, '</s></s>')
    elif ('</s>' in w):
        w = [w.replace('</s>', ''), '</s>']
    else:
        w = [w]
    words.extend(w)

In [51]:
words_

['<s>Man',
 'and',
 'woman',
 'rowing',
 'on<mask>boards',
 'third',
 'rower',
 'in',
 'distance.</s></s>Couple',
 'going',
 'fishing.</s>']

In [50]:
words

['<s>',
 'Man',
 'and',
 'woman',
 'rowing',
 'on<mask>boards',
 'third',
 'rower',
 'in',
 'distance.',
 '</s></s>',
 'Couple',
 'going',
 'fishing.',
 '</s>']